___
## Ciência dos Dados - PROJETO 3 - INSPER

___
#### Freddy Dratwa

#### Victor Vazquez

#### Michel Haggiag

#### Allan Singer

#### Ricardo Ferraro

___

# Estudo do preço do carro

___

## A. INTRODUÇÃO

## https://www.kaggle.com/jpayne/852k-used-car-listings
## https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html

O objetivo deste projeto é chegar ao melhor modelo possível que seja capaz de prever o preço de um carro usado nos EUA a partir das características presentes no dataset:
    <br>- Preço;<br>
    - Ano;<br>
    - Quilometragem;<br>
    - Cidade;<br>
    - Estado;<br>
    - Marca;<br>
    - Modelo;<br>
    - Número de Chassis.<br>

A base de dados possui 1.2 milhão de carros com data de produção entre os anos 1997 a 2018 de 58 marcas diferentes, sendo estes 2914 modelos distintos com o preço mÍnimo de US1.500,00 e máximo de US462.000,00 , com a mínima quilometragem 8 Km e a máxima 2.286.260,8 Km.<br>

Como uma primeira previsão, imagina-se que algumas das características do dataset influenciam mais no preço do carro do que outras. Por exemplo, é de se esperar que um carro comprado há mais tempo tenha seu preço menor atualmente. Ou seja, a variavel "Ano", que representa o ano em que o carro foi fabricado, deve influenciar muito no preço do veículo, bem como a quilometragem, onde um carro que foi comprado mas usado pouco deve permanecer mais barato do que um usado muitas vezes.<br>

Pelo outro lado, imagina-se que a variável "Cidade", a qual representa a cidade onde o carro foi comprado, não tenha tanta influência no preço do veículo, a não ser que comparadas  duas cidades com condições muito diferentes. Por exemplo, como se tratam de cidades apenas nos EUA, imaina-se que em cidades no litoral, os carros se depreciem com velocidade maior, como consequência do fenômeno da Maresia.<br>

___
## B. MINERANDO DADOS e CARACTERÍSTICAS DO DATASET

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split

### BASE DE DADOS

**Limpando e tratando dataset**

Nas células abaixo, o dataset é limpo e reorganiazdo. Para se fazer isso, a coluna "Chassi" é removoida, pois esta é uma característica individual para cada veículo e não influencia no preço final do mesmo. Alem disso, duas colunas vazias foram reitradas. Então, a variável milhagem foi transformada em quilometragem, multplicando seu valor por 1.6. Por último, a variavel ano foi transformada em idade, ou seja, a diferença do ano atual (2019) para o ano em que o carro foi vendido. 

In [3]:
dataset = pd.read_csv("Dataframe.csv")

In [4]:
dataset.head()

,Unnamed: 0,ID,PRECO,ANO,IDADE,KM,CIDADE,ESTADO,MARCA,MODELO
0,0,516508.0,32500.0,2015.0,4.0,35886.4,ANKENY,IA,CHEVROLET,ColoradoCrew
1,1,744220.0,43310.0,2017.0,2.0,5918.4,MIAMI,FL,GMC,AcadiaFWD
2,2,341109.0,26887.0,2016.0,3.0,20011.2,HOUSTON,TX,TOYOTA,SiennaL
3,3,709465.0,34999.0,2014.0,5.0,49478.4,BOWLING GREEN,KY,FORD,F-150FX4
4,4,841923.0,27198.0,2014.0,5.0,106585.6,LAS CRUCES,NM,INFINITI,QX60AWD


## ANÁLISE DESCRITIVA

___
## C. MODELOS DE PREDIÇÃO

o MODELO DE PREDIÇÃO PELA MÉDIA (Sem uso de variável explicativa).

**Separando a base de dados em treinamento e teste aleatoriamente**

In [5]:
train, test = train_test_split(dataset, test_size=0.25, train_size=0.75)

In [6]:
y_train = train["PRECO"].mean()
y_test = test["PRECO"].mean() # contem o preco previsto de um novo carro da base teste pelo moedelo de medias

In [7]:
rms_train = []
rms_test = []
num_train = []
dem_train = []
num_test = []
dem_test = []
##### ARRRRRRRUMAR OS R SQUARED VIDE FOTO 
for a in train["PRECO"]:
    rms_train.append(np.sqrt((a-y_train)**2))
    num_train.append((a-y_train)**2)
    dem_train.append((a-y_train)**2)
for e in test["PRECO"]:
    rms_test.append(np.sqrt((e-y_train)**2))
    num_test.append((e-y_train)**2)
    dem_test.append((e-y_test)**2)

RMS_train_M = np.mean(rms_train)
RMS_test_M = np.mean(rms_test)
R_train_M = 1-(sum(num_train)/sum(dem_train))
R_test_M = 1-(sum(num_test)/sum(dem_train))
R_train = 1-(sum(num_train)/sum(dem_train))


In [8]:
R_train_M,RMS_train_M

(0.0, 8979.677560192)

In [9]:
R_test_M, RMS_test_M

(0.6846635925416844, 8932.445665728)

In [10]:
R_perc_M = 0
RMS_perc_M = ((RMS_test_M-RMS_train_M)/RMS_train_M)*100

o MODELO DOS K VIZINHOS MAIS PRÓXIMOS (K-Nearest Neighbors Regression)

o MODELO DE REGRESSÃO LINEAR (Multiple Linear Regression)

In [3]:
def dummify(data, column_name):
    return pd.get_dummies(data[column_name],drop_first = False, prefix = column_name)

def regress(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    ypred = results.predict(X_cp)
    return results, ypred

In [5]:
final = pd.concat([dummify(dataset, "ESTADO").dropna(), dummify(dataset, "MODELO").dropna(), dummify(dataset, "MARCA").dropna()], axis=1)

In [7]:
dt = dataset
dt = dt.drop(columns = ["ID", "ANO", "CIDADE", "ESTADO", "MARCA", "MODELO"])
final = pd.concat([dt, final], axis = 1)
final.head()

In [8]:
df = final

x = df[np.delete(df.columns.values, 0)]
y = df[["PRECO"]]

results = regress(x,y)

dataresultados = pd.read_html(results[0].summary().tables[1].as_html(), header=0, index_col=0)[0]

In [6]:
dataresultados.head(10)

In [0]:
df = final
for i in range(5):
    df.drop(columns = dataresultados.loc[(dataresultados["P>|t|"] > 0.10)].index)
    x = df[np.delete(df.columns.values, 0)]
    y = df[["PRECO"]]

    results = regress(x,y)

    dataresultados = pd.read_html(results[0].summary().tables[1].as_html(), header=0, index_col=0)[0]

quanti = 0
for i in dataresultados["P>|t|"]:
    if i > 0.10:
        quanti+=1
print(quanti)

o MODELO DE ÁRVORES DE REGRESSÃO (Decision Tree Regression)

___
## D. PROCESSO E ESTATÍSTICAS DE VALIDAÇÃO

Nesta céluka abaixo foi criado um dataframe com os valores de Rˆ2 e RMS de treinamento e de teste de cada variável, além disso o percentual do Rˆ2 e do RMS.

In [11]:
predicoes = ["Media", "K-NN", "Regressão", "Árvore de Decisão"]
R_TRAIN = [R_train_M, 0, 0, 0]
R_TEST = [R_test_M, 0, 0, 0]
R_PERC = [R_perc_M, 0, 0, 0]
RMS_TRAIN = [RMS_train_M, 0, 0, 0]
RMS_TEST = [RMS_test_M, 0, 0, 0]
RMS_PERC = [RMS_perc_M, 0, 0, 0]
PREDI = pd.DataFrame(data = predicoes)
PREDI.columns = ["Modelo"]
PREDI["Rˆ2 Treinamento"] = R_TRAIN
PREDI["Rˆ2 Teste"] = R_TEST
PREDI["Rˆ2 %"] = R_PERC
PREDI["RMS Treinamento"] = RMS_TRAIN
PREDI["RMS Teste"] = RMS_TEST
PREDI["RMS %"] = RMS_PERC

In [12]:
PREDI

,Modelo,Rˆ2 Treinamento,Rˆ2 Teste,Rˆ2 %,RMS Treinamento,RMS Teste,RMS %
0,Media,0.0,0.684664,0,8979.67756,8932.445666,-0.525987
1,K-NN,0.0,0.000000,0,0.00000,0.000000,0.000000
2,Regressão,0.0,0.000000,0,0.00000,0.000000,0.000000
3,Árvore de Decisão,0.0,0.000000,0,0.00000,0.000000,0.000000


___
## E. CONCLUSÃO

___
## F. REFERÊNCIAS BIBLIOGRÁFICAS